In [1]:
import keras
from sklearn.datasets import fetch_lfw_people
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [2]:
#getting data
lfw_people = fetch_lfw_people(min_faces_per_person=50, resize=0.5)
X = lfw_people.data
y = lfw_people.target

In [3]:
#splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [4]:
## scale the data
mms = MinMaxScaler()
X_train_scaled = mms.fit_transform(X_train)
X_test_scaled = mms.fit_transform(X_test)

In [5]:
#dnn model definition
dnn_model = keras.Sequential([
    keras.layers.Dense(8, activation='relu', input_shape=[X_train.shape[1]]),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(len(set(y_train)), activation='softmax')
    ])

In [6]:
#compiling the data
dnn_model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.003), metrics=['accuracy'])
dnn_model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.3)

Epoch 1/100


C:\Users\ethan\PycharmProjects\EET4501\venv\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


22/22 [==============================] - 1s 12ms/step - loss: 2.0552 - accuracy: 0.3234 - val_loss: 1.9885 - val_accuracy: 0.3821
Epoch 2/100
22/22 [==============================] - 0s 3ms/step - loss: 1.9928 - accuracy: 0.3689 - val_loss: 1.8943 - val_accuracy: 0.3821
Epoch 3/100
22/22 [==============================] - 0s 3ms/step - loss: 1.9202 - accuracy: 0.3775 - val_loss: 1.8889 - val_accuracy: 0.4186
Epoch 4/100
22/22 [==============================] - 0s 2ms/step - loss: 1.8695 - accuracy: 0.3718 - val_loss: 1.8135 - val_accuracy: 0.4153
Epoch 5/100
22/22 [==============================] - 0s 2ms/step - loss: 1.7441 - accuracy: 0.4231 - val_loss: 1.6639 - val_accuracy: 0.4551
Epoch 6/100
22/22 [==============================] - 0s 2ms/step - loss: 1.6587 - accuracy: 0.4573 - val_loss: 1.8097 - val_accuracy: 0.4684
Epoch 7/100
22/22 [==============================] - 0s 2ms/step - loss: 1.6500 - accuracy: 0.4587 - val_loss: 1.5797 - val_accuracy: 0.4950
Epoch 8/100
22/22 [=====

In [7]:
#determine the dnn score
dnn_score = dnn_model.evaluate(X_test_scaled, y_test)

14/14 [==============================] - 0s 878us/step - loss: 2.0441 - accuracy: 0.4478


In [8]:
#cnn model definition
cnn_model = keras.Sequential([
    keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(124, 94, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(len(set(y_train)), activation='softmax')
])

In [9]:
cnn_model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.003), metrics=['accuracy'])

In [10]:
#reshaping the data
X_train_reshape = X_train_scaled.reshape(X_train.shape[0], 62, 47, 1)
X_test_reshape = X_test_scaled.reshape(X_test.shape[0], 62, 47, 1)
X_train_reshape = tf.image.resize(X_train_reshape, (124, 94))
X_test_reshape = tf.image.resize(X_test_reshape, (124, 94))

In [11]:
cnn_model.fit(X_train_reshape, y_train, epochs=100, validation_split=0.3)

Epoch 1/100
22/22 [==============================] - 1s 35ms/step - loss: 2.0571 - accuracy: 0.3476 - val_loss: 1.7555 - val_accuracy: 0.4120
Epoch 2/100
22/22 [==============================] - 1s 34ms/step - loss: 1.5730 - accuracy: 0.4573 - val_loss: 1.3839 - val_accuracy: 0.5150
Epoch 3/100
22/22 [==============================] - 1s 34ms/step - loss: 1.0833 - accuracy: 0.6695 - val_loss: 1.0800 - val_accuracy: 0.6777
Epoch 4/100
22/22 [==============================] - 1s 32ms/step - loss: 0.6572 - accuracy: 0.8291 - val_loss: 0.9861 - val_accuracy: 0.6944
Epoch 5/100
22/22 [==============================] - 1s 34ms/step - loss: 0.4459 - accuracy: 0.9003 - val_loss: 0.7619 - val_accuracy: 0.7342
Epoch 6/100
22/22 [==============================] - 1s 32ms/step - loss: 0.3365 - accuracy: 0.9145 - val_loss: 0.8384 - val_accuracy: 0.7409
Epoch 7/100
22/22 [==============================] - 1s 32ms/step - loss: 0.2440 - accuracy: 0.9587 - val_loss: 0.6815 - val_accuracy: 0.7741
Epoch 

In [12]:
#getting cnn score
cnn_score = cnn_model.evaluate(X_test_reshape, y_test)

14/14 [==============================] - 0s 9ms/step - loss: 0.8335 - accuracy: 0.8190


In [13]:
#printing the cnn and dnn scores
print(f'dnn accuracy is {dnn_score[1]}')
print(f'cnn accuracy is {cnn_score[1]}')

dnn accuracy is 0.4477958381175995
cnn accuracy is 0.8190255165100098


1. For CNN, when the learning rate is increased, the accuracy drops. When the filter is increased it also decreases the accuracy.
2. Decreasing the hidden nodes in DNN significantly reduces the accuracy of the model